## Конкурс по анализу тональности текста - Стрижов Григорий - CS

Я начал с того, что попробовал нейросеть над самыми частотными словами. А точнее не с этого, а с того, что добавил параметр quoting=csv.QUOTE_NONE в csv.reader из baseline. Если этого не сделать, то некоторые объекты обучающей выборки будут слипаться в один из-за неправильно расставленных кавычек.

С нейросетями мне удалось добиться точности 0.56. Я использовал стоп-слова и стеммер из nltk. Без них точность была ещё меньше.

После чего я стал гуглить и искать как решают подобные задачи на kaggle. Так я наткнулся на хорошую идею - использовать не частотные слова, а TfidfVectorizer. Я знал об этой штуке и раньше, но гугл помог вспомнить. Признаков в векторах получается достаточно много, поэтому я не придумал как запихнуть их в нейросеть и решил ограничиться более простыми моделями. Начал с того, что обучил логистическую регрессию с tfidf векторизацией, в которой использовались только униграммы. Таким образом я получил точность 0.58. 

Я попробовал не использовать стоп-слова и стеммер и обнаружил, что от стеммера вообще ничего не зависит, а без стоп-слов качество становится чуть лучше. После такого наблюдения я перестал как либо предобрабатывать текст и решил повысить "граммность" признаков. 

Времени до конца конкурса уже оставалось очень мало, а подозрения, что количество грамм может что-то улучшить были. Поскольку памяти у меня много, то я решил сразу взять триграммы (1, 2 и 3-граммы. 4-граммы не рискнул). 

Лучший результат дала логистическая регрессия с параметрами C=3.25 и class_weight='balanced'. Качество классификации было 0.605. Кроме неё я пробовал Наивный Байес и линейный SVM. Байес дал результат почти на 2% хуже, а SVMа я так и не дождался.

В итоге я решил успокоиться, т.к. был в целом доволен результатом 0.605. Осталось только настекать модель!

Я разбил обучающую выборку на 5 частей. После чего на 4х частях учился, а для пятой делал предсказание вероятности с помощью уже обученной модели. И так для каждой 1/5 выборки. Для теста я просто обучился на train, и сделал предсказание. В итоге я получил 5 новых признаков для каждого объекта выборки.

Над новыми признаками я решил обучить что-нибудь нелинейное, поскольку композиция линейных моделей это линейная модель. В итоге выбор был между XGboost и RandomForest. XGboost требует более тонкой настройки и более чувствителен к шумам, поэтому я решил положиться на RandomForest. Итого я имел выборку с векторами длины 5 и RandomForest. Как я ни старался, а результат при стекинге не улучшился ни на %. Что можно сделать в таком случае? Ну, например, добавить признаков. Я решил взять квадратичные признаки, использовав sklearn.preprocessing.PolynomialFeatures. Первую координату я отрезал, т.к. она у всех векторов одинаковая и равна 1 (нулевые степени). Получились векторы длины 20. И вот над ними стекинг дал прирост 1.2 %. Итоговая точность на кроссвалидации была 0.617096566649. Параметры RandomForest: 1000 деревьев (я решил зафиксировать побольше и подгоняться под это количество для стабильности), максимальная глубина: 8, 'max_features': 3.

В итоге я занял 7е место на public и на private. Точность на private: 0.62684. На паблике: 0.62607.

In [109]:
import nltk
import numpy as np
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import normalize
from scipy.sparse import vstack
import pickle
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestClassifier

In [89]:
def serialize(data, filename):
    with open(filename, 'wb') as pklfile:
        pickle.dump(data, pklfile)
        
        
def get_serialized_data(filename):
    with open(filename, 'rb') as pklfile:
        return pickle.load(pklfile)

In [2]:
train_file = csv.reader(open('train.data'), delimiter='\t', quoting=csv.QUOTE_NONE)
next(train_file)
train_set = [x for x in train_file]
test_file = csv.reader(open('test.data'), delimiter='\t', quoting=csv.QUOTE_NONE)
next(test_file)
test_set = [x for x in test_file]

In [3]:
train_data, train_label = [line[2] for line in train_set], [line[1] for line in train_set]
test_data = [line[1] for line in test_set]

In [4]:
len(train_data)

102582

In [5]:
x_train_and_test = TfidfVectorizer(ngram_range=(1, 3), min_df=3).fit_transform(train_data + test_data)

In [6]:
x_train_and_test = normalize(x_train_and_test)

In [7]:
x_train = x_train_and_test[:102582]
x_test = x_train_and_test[102582:]

In [8]:
print(x_train.shape)
print(x_test.shape)

(102582, 1825915)
(34194, 1825915)


In [9]:
print(x_train_and_test.shape)

(136776, 1825915)


In [10]:
h#from sklearn.cross_validation import train_test_split
#x_train, x_validate, y_train, y_validate = train_test_split(x_train, train_label, test_size=0.5, random_state=42)

In [11]:
test_id = [x[0] for x in test_set]

In [12]:
parameters = {'C': np.logspace(-5, 10, 50), 'class_weight': ['balanced']}
clf = GridSearchCV(LogisticRegression(), parameters, verbose=50, scoring='accuracy', refit=False, cv=5, n_jobs=-1)
clf.fit(x_train, train_label)
print(clf.best_score_)
print(clf.best_estimator_)x_train

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] C=1e-05, class_weight=balanced ..................................
[CV] C=1e-05, class_weight=balanced ..................................
[CV] C=1e-05, class_weight=balanced ..................................
[CV] C=1e-05, class_weight=balanced ..................................
[CV] C=1e-05, class_weight=balanced ..................................
[CV] C=2.02358964773e-05, class_weight=balanced ......................
[CV] C=2.02358964773e-05, class_weight=balanced ......................
[CV] C=2.02358964773e-05, class_weight=balanced ......................
[CV]  C=2.02358964773e-05, class_weight=balanced, score=0.345485 -   0.5s
[CV] C=2.02358964773e-05, class_weight=balanced ......................
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   55.0s
[CV]  C=2.02358964773e-05, class_weight=balanced, score=0.345486 -   0.4s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   55.5s
[CV] C=2.02358964773e-05, c

KeyboardInterrupt: 

In [ ]:
C=3.23745754282 0.608335

In [21]:
parameters = {'C': [3.25], 'class_weight': ['balanced']}
clf = GridSearchCV(LogisticRegression(), parameters, verbose=50, scoring='accuracy', refit=False, cv=5, n_jobs=-1)
clf.fit(x_train, train_label)
print(clf.best_params_)
print(clf.best_score_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] C=3.25, class_weight=balanced ...................................
[CV] C=3.25, class_weight=balanced ...................................
[CV] C=3.25, class_weight=balanced ...................................
[CV] C=3.25, class_weight=balanced ...................................
[CV] C=3.25, class_weight=balanced ...................................
[CV] .......... C=3.25, class_weight=balanced, score=0.603100 -   0.2s
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  4.6min
[CV] .......... C=3.25, class_weight=balanced, score=0.607623 -   0.3s
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  4.6min remaining:  6.9min
[CV] .......... C=3.25, class_weight=balanced, score=0.608335 -   0.2s
[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:  4.6min remaining:  3.1min
[CV] .......... C=3.25, class_weight=balanced, score=0.600985 -   0.2s
[CV] .......... C=3.25, class_weight=balanced, score=0.606220 -   0.1s
[Parall

In [17]:
print(clf.best_params_)
print(clf.best_score_)

{'C': 3.2, 'class_weight': 'balanced'}
0.60515490047


In [22]:
print(clf.best_params_)
print(clf.best_score_)

{'C': 3.25, 'class_weight': 'balanced'}
0.605252383459


In [23]:
x_train.shape

(102582, 1825915)

In [27]:
fifth = 102582 // 5

In [56]:
x_train1 = x_train[:fifth]
train_label1 = train_label[:fifth]

x_train2 = x_train[fifth:2*fifth]
train_label2 = train_label[fifth:2*fifth]

x_train3 = x_train[2*fifth:3*fifth]
train_label3 = train_label[2*fifth:3*fifth]

x_train4 = x_train[3*fifth:4*fifth]
train_label4 = train_label[3*fifth:4*fifth]

x_train5 = x_train[4*fifth:]
train_label5 = train_label[4*fifth:]

In [35]:
clf = LogisticRegression(C=3.25, class_weight='balanced')

In [62]:
vstack([x_train1, x_train2, x_train3, x_train4, x_train5])

(102582, 1825915)

In [65]:
np.concatenate((train_label1, train_label2, train_label3, train_label4, train_label5), axis=0).shape

(102582,)

In [81]:
clf.fit(vstack([x_train2, x_train3, x_train4, x_train5]), 
        np.concatenate((train_label2, train_label3, train_label4, train_label5), axis=0))
pred1 = clf.predict_proba(x_train1)

In [82]:
clf.fit(vstack([x_train1, x_train3, x_train4, x_train5]), 
        np.concatenate((train_label1, train_label3, train_label4, train_label5), axis=0))
pred2 = clf.predict_proba(x_train2)

In [83]:
clf.fit(vstack([x_train1, x_train2, x_train4, x_train5]), 
        np.concatenate((train_label1, train_label2, train_label4, train_label5), axis=0))
pred3 = clf.predict_proba(x_train3)

In [84]:
clf.fit(vstack([x_train1, x_train2, x_train3, x_train5]), 
        np.concatenate((train_label1, train_label2, train_label3, train_label5), axis=0))
pred4 = clf.predict_proba(x_train4)

In [85]:
clf.fit(vstack([x_train1, x_train2, x_train3, x_train4]), 
        np.concatenate((train_label1, train_label2, train_label3, train_label4), axis=0))
pred5 = clf.predict_proba(x_train5)

In [92]:
new_x_train = np.concatenate((pred1, pred2, pred3, pred4, pred5), axis=0)
print(new_x_train.shape)

(102582, 5)


In [93]:
clf.fit(x_train, train_label)

LogisticRegression(C=3.25, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [95]:
new_x_test = clf.predict_proba(x_test)
print(new_x_test.shape)

(34194, 5)


In [98]:
#serialize(new_x_train, 'new_x_train.pkl')
#serialize(new_x_test, 'new_x_test.pkl')

In [100]:
new_x_all = np.concatenate((new_x_train, new_x_test), axis=0)
print(new_x_all.shape)

(136776, 5)


In [123]:
poly = PolynomialFeatures(2)
sq_x_all = poly.fit_transform(new_x_all)
print(sq_x_all.shape)

(136776, 21)


In [124]:
sq_x_all = sq_x_all[:, 1:]
print(sq_x_all.shape)

(136776, 20)


In [125]:
sq_x_train = sq_x_all[:102582]
sq_x_test = sq_x_all[102582:]

In [126]:
#serialize(sq_x_train, 'sq_x_train.pkl')
#serialize(sq_x_test, 'sq_x_test.pkl')

In [132]:
parameters = {'n_estimators':[100], 'random_state':[42], 'n_jobs':[-1], 'max_depth':[8, 10, 11, 12, 13, 14, 15]}#, 'max_features':[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]}
clf = GridSearchCV(RandomForestClassifier(), parameters, verbose=50, scoring='accuracy', refit=False, cv=5, n_jobs=-1)
clf.fit(sq_x_train, train_label)
print(clf.best_params_)
print(clf.best_score_)

Fitting 5 folds for each of 7 candidates, totalling 35 fits
[CV] n_jobs=-1, random_state=42, max_depth=8, n_estimators=100 .......
[CV] n_jobs=-1, random_state=42, max_depth=8, n_estimators=100 .......
[CV] n_jobs=-1, random_state=42, max_depth=8, n_estimators=100 .......
[CV] n_jobs=-1, random_state=42, max_depth=8, n_estimators=100 .......
[CV] n_jobs=-1, random_state=42, max_depth=8, n_estimators=100 .......
[CV] n_jobs=-1, random_state=42, max_depth=10, n_estimators=100 ......
[CV] n_jobs=-1, random_state=42, max_depth=10, n_estimators=100 ......
[CV] n_jobs=-1, random_state=42, max_depth=10, n_estimators=100 ......
[CV]  n_jobs=-1, random_state=42, max_depth=8, n_estimators=100, score=0.609776 -   0.5s
[CV] n_jobs=-1, random_state=42, max_depth=10, n_estimators=100 ......
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   28.7s
[CV]  n_jobs=-1, random_state=42, max_depth=8, n_estimators=100, score=0.621009 -   0.3s
[CV] n_jobs=-1, random_state=42, max_depth=10, n_estimators=1

In [133]:
parameters = {'n_estimators':[100], 'random_state':[42], 'n_jobs':[-1], 'max_depth':[8], 'max_features':[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]}
clf = GridSearchCV(RandomForestClassifier(), parameters, verbose=50, scoring='accuracy', refit=False, cv=5, n_jobs=-1)
clf.fit(sq_x_train, train_label)
print(clf.best_params_)
print(clf.best_score_)

Fitting 5 folds for each of 19 candidates, totalling 95 fits
[CV] n_jobs=-1, random_state=42, max_depth=8, n_estimators=100, max_features=2 
[CV] n_jobs=-1, random_state=42, max_depth=8, n_estimators=100, max_features=2 
[CV] n_jobs=-1, random_state=42, max_depth=8, n_estimators=100, max_features=2 
[CV] n_jobs=-1, random_state=42, max_depth=8, n_estimators=100, max_features=2 
[CV] n_jobs=-1, random_state=42, max_depth=8, n_estimators=100, max_features=2 
[CV] n_jobs=-1, random_state=42, max_depth=8, n_estimators=100, max_features=3 
[CV] n_jobs=-1, random_state=42, max_depth=8, n_estimators=100, max_features=3 
[CV] n_jobs=-1, random_state=42, max_depth=8, n_estimators=100, max_features=3 
[CV]  n_jobs=-1, random_state=42, max_depth=8, n_estimators=100, max_features=2, score=0.617214 -   0.5s
[CV] n_jobs=-1, random_state=42, max_depth=8, n_estimators=100, max_features=3 
[CV]  n_jobs=-1, random_state=42, max_depth=8, n_estimators=100, max_features=2, score=0.610264 -   0.3s
[CV] n_jo

In [137]:
parameters = {'n_estimators':[1000, 2000], 'random_state':[42], 'n_jobs':[-1], 'max_depth':[8], 'max_features':[4]}
clf = GridSearchCV(RandomForestClassifier(), parameters, verbose=50, scoring='accuracy', refit=False, cv=5, n_jobs=-1)
clf.fit(sq_x_train, train_label)
print(clf.best_params_)
print(clf.best_score_)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] n_jobs=-1, random_state=42, max_depth=8, n_estimators=1000, max_features=4 
[CV] n_jobs=-1, random_state=42, max_depth=8, n_estimators=1000, max_features=4 
[CV] n_jobs=-1, random_state=42, max_depth=8, n_estimators=1000, max_features=4 
[CV] n_jobs=-1, random_state=42, max_depth=8, n_estimators=1000, max_features=4 
[CV] n_jobs=-1, random_state=42, max_depth=8, n_estimators=1000, max_features=4 
[CV] n_jobs=-1, random_state=42, max_depth=8, n_estimators=2000, max_features=4 
[CV] n_jobs=-1, random_state=42, max_depth=8, n_estimators=2000, max_features=4 
[CV] n_jobs=-1, random_state=42, max_depth=8, n_estimators=2000, max_features=4 
[CV]  n_jobs=-1, random_state=42, max_depth=8, n_estimators=1000, max_features=4, score=0.610361 -   3.2s
[CV] n_jobs=-1, random_state=42, max_depth=8, n_estimators=2000, max_features=4 
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  4.8min
[CV]  n_jobs=-1, random_state=42, max_depth

In [144]:
parameters = {'n_estimators':[100], 'random_state':[42], 'n_jobs':[-1], 'max_depth':[8], 'max_features':[4]}
clf = GridSearchCV(RandomForestClassifier(), parameters, verbose=50, scoring='accuracy', refit=False, cv=5, n_jobs=-1)
clf.fit(sq_x_train, train_label)
print(clf.best_params_)
print(clf.best_score_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] n_jobs=-1, random_state=42, max_depth=8, n_estimators=100, max_features=4 
[CV] n_jobs=-1, random_state=42, max_depth=8, n_estimators=100, max_features=4 
[CV] n_jobs=-1, random_state=42, max_depth=8, n_estimators=100, max_features=4 
[CV] n_jobs=-1, random_state=42, max_depth=8, n_estimators=100, max_features=4 
[CV] n_jobs=-1, random_state=42, max_depth=8, n_estimators=100, max_features=4 
[CV]  n_jobs=-1, random_state=42, max_depth=8, n_estimators=100, max_features=4, score=0.609776 -   0.3s
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   18.0s
[CV]  n_jobs=-1, random_state=42, max_depth=8, n_estimators=100, max_features=4, score=0.618530 -   0.3s
[CV]  n_jobs=-1, random_state=42, max_depth=8, n_estimators=100, max_features=4, score=0.621009 -   0.3s
[CV]  n_jobs=-1, random_state=42, max_depth=8, n_estimators=100, max_features=4, score=0.616506 -   0.3s
[CV]  n_jobs=-1, random_state=42, max_depth=8, n_estimators

In [145]:
parameters = {'n_estimators':[1000], 'random_state':[42], 'n_jobs':[-1], 'max_depth':[8], 'max_features':[2, 3, 4, 5, 6, 7, 8, 9, 10]}
clf = GridSearchCV(RandomForestClassifier(), parameters, verbose=50, scoring='accuracy', refit=False, cv=5, n_jobs=-1)
clf.fit(sq_x_train, train_label)
print(clf.best_params_)
print(clf.best_score_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] n_jobs=-1, random_state=42, max_depth=8, n_estimators=1000, max_features=2 
[CV] n_jobs=-1, random_state=42, max_depth=8, n_estimators=1000, max_features=2 
[CV] n_jobs=-1, random_state=42, max_depth=8, n_estimators=1000, max_features=2 
[CV] n_jobs=-1, random_state=42, max_depth=8, n_estimators=1000, max_features=2 
[CV] n_jobs=-1, random_state=42, max_depth=8, n_estimators=1000, max_features=2 
[CV] n_jobs=-1, random_state=42, max_depth=8, n_estimators=1000, max_features=3 
[CV] n_jobs=-1, random_state=42, max_depth=8, n_estimators=1000, max_features=3 
[CV] n_jobs=-1, random_state=42, max_depth=8, n_estimators=1000, max_features=3 
[CV]  n_jobs=-1, random_state=42, max_depth=8, n_estimators=1000, max_features=2, score=0.618676 -   3.6s
[CV] n_jobs=-1, random_state=42, max_depth=8, n_estimators=1000, max_features=3 
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  2.9min
[CV]  n_jobs=-1, random_state=42, max_depth

In [149]:
clf = RandomForestClassifier(n_estimators=1000, random_state=42, n_jobs=-1,
                             max_depth=8, max_features=3)
clf.fit(sq_x_train, train_label)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=8, max_features=3, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=-1, oob_score=False, random_state=42,
            verbose=0, warm_start=False)

In [151]:
prediction = clf.predict(sq_x_test)

In [154]:
writer = csv.writer(open('stacking_logreg.csv', 'w'))
writer.writerow(['ID', 'Sentiment'])
for id_, prediction in zip(test_id, prediction):
    writer.writerow([id_, prediction])